# Text Classification with Supervised Machine Learning

The goal of text classification is to classify texts into any number of predefined categories. This method is most similar to traditional content analysis, or text coding, in that it does the same thing as a team of trained undergrads: place texts into categories. 

It does require labeled text, or text that is already categorized into predefined categories. In some cases, like ours today, this is built into the data. Labeling text can also be done by hand. Once we have a good number of labeled texts, usually between 200 and 500, we can use superivsed machine learning algorithms to train a computer to recognize the categories and place the remaining, un-coded texts into a category. This method has two benefits: (1) It allows us to scale our hand-coding up almost indefinitely, and (2) it identifies what *features* (in our case, words), are most defining of each category. This can help us learn more about the content of our categories.

This lesson is a basic, non-mathematical introduction to supervised machine learning. I encourage everyone to learn more about the math behind these techniques. But you can successfully implement them and use them for research with the basic skills presented here.

### Learning Goals
* Get you comfortable with the basic vocabulary around supervised machine learning and text classification
* Understand the intuition behind supervised machine learning
* Learn how to implement a few key supervised machine learning algorithms
* Understand how to test for accuracy
* Use scikit-learn to identify important features for each category
* Be equipped with foundational knowledge so you can continue to learn, either on your own or by taking more advanced math, machine learning, or text analysis courses

### Outline

0. Reading in and pre-processing data
1. Training and test sets
2. Supervised Machine Learning Classification
4. Cross Validation
5. Identifying Features


### Key Terms

* *supervised machine learning* :
    * a machine learning task of inferring a classification from labeled training data.
* *features*:
    * way of representing the object that will be classified. For images, features are often pixels. For text, features are usually word counts or weighted word counts, but they also also be things like a word's part of speech.
* *training set*:
    * a selection of labeled data that is used to train the machine learning algorithm
* *test set*:
    * a selection of labeled data that is used to test the accuracy of the machine learning algorithm
* *accuracy*:
    * the number of texts the algorithms correctly classifies
* *cross validation*:
    * a way to assess how the algorithm will perform on an unseen data set
    
### Additional Resources:

The [full documentation](http://scikit-learn.org/stable/supervised_learning.html) for supervised machine learning using scikit-learn

[Identifying what types of blog posts are censored in China](http://gking.harvard.edu/publications/how-censorship-china-allows-government-criticism-silences-collective-expression), using supervised machine learning, Gary King, Jennifer Pan, and Margaret E Roberts


[Literary Pattern Recognition](https://lucian.uchicago.edu/blogs/literarynetworks/files/2015/12/LONG_SO_CI.pdf), Hoyt Long, Richard So

[How Quickly Do Literary Standards Change?](https://tedunderwood.com/2015/05/18/how-quickly-do-literary-standards-change/), Ted Underwood, Jordan Sellers


### 0. Reading in and pre-processing data

In [ ]:
#first, import the necessary modules
import pandas
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC #Linear Suppot Vector Classifier
from sklearn.naive_bayes import MultinomialNB #Naive Bayes classifier
from sklearn.metrics import accuracy_score #to asses the accuracy of the algorithm
from sklearn.model_selection import cross_val_score #to compute cross validation for assessment purposes

In [ ]:
#Read in our dataset. Remeber, the column with the review is 'body'
df_full = pandas.read_csv("../A-Data/BDHSI2016_music_reviews.csv", sep = '\t', encoding = 'utf-8')
df = df_full[(df_full['genre']=='Rap') | (df_full['genre']=='Country')]
df['body'] = df['body'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
df

### 1. Divide data into training and test sets

First we need to create a training set and a test set. We'll train on the first 300 reviews, and test the accuracy on the last 200.

In [ ]:
df = df.sample(503, random_state=1)
df

In [ ]:
df_train = df[:300]
df_test = df[300:]
df_test

### 2. Supervised Machine Learning Classification

Next we need to create a dtm for each review, and an array containing the classification label for each review (for us, this is genre)

In [ ]:
#transform the 'body' column into a document term matrix
tfidfv = TfidfVectorizer(stop_words = 'english', min_df = 1, binary=True)
cv = CountVectorizer(stop_words = 'english', min_df = 1, binary=True)

training_dtm = cv.fit_transform(df_train.body)
test_dtm = cv.transform(df_test.body)

#create an array for labels
training_labels = df_train.genre
test_labels = df_test.genre

In [ ]:
#note there is not output for this cell, we're training, fitting, and predicting the model only
nb = MultinomialNB()
nb.fit(training_dtm, training_labels)
predictions = nb.predict(test_dtm) 

We can use the built-in function "accuracy-score" to calculate the accuracy of our classifier. For binary and multiclass classification, which is our case, this function calculates Jaccard similarity coefficient, defined as the size of the intersection divided by the size of the union of two label sets. It is used to compare the set of predicted labels (the labels the algorithms assigned to the test set) to the true labels for the test set.

In [ ]:
accuracy_score(predictions, test_labels)

In [ ]:
####Ex: Train a Linear SVC instead of a multinomial naive bayes classifier.
#       Hint: Look at the Import statements above

### 3. Cross Validation

We know the accuracy for the test set we created, but what if we have a bunch of reviews that are not yet classified and we want to classify them? Can we be sure we'll get a similar accuracy?

In [ ]:
dtm = cv.fit_transform(df.body)
scores = cross_val_score(nb, dtm, df.genre, cv=5)
scores   

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

*NOTE: This assumes any unseen data has similar features as your training and test data. If the features are different, you can not assume you will get similar accuracy. When might the unseen data be different from the labeled data?

Ex: Why is the cross validation accuracy lower than our classifier? 

Turn to the person next to you and talk through the above code. What is happening in each line?

### 5. Identifying Features

We can use supervised machine learning to classify unseen documents using the above code. But we can also use it to learn more about the content of each category, by extracting the most defining features of this category. So even if we do not have unseen text, we can use this method to better understand given categories (for example, canonized and non-canonized text). 

In [ ]:
# Hand-waving the underlying statistics here...

def most_informative_features(text_class, vectorizer = cv, classifier = nb, top_n = 20):

    feature_names = vectorizer.get_feature_names()
    class_index = np.where(classifier.classes_==(text_class))[0][0]
    
    class_prob_distro = np.exp(classifier.feature_log_prob_[class_index])
    alt_class_prob_distro = np.exp(classifier.feature_log_prob_[1 - class_index])
    
    odds_ratios = class_prob_distro / alt_class_prob_distro
    odds_with_fns = sorted(zip(odds_ratios, feature_names), reverse = True)
    
    return odds_with_fns[:top_n]

In [ ]:
# Returns feature name and odds ratio for a given class
most_informative_features('Rap')

In [ ]:
most_informative_features('Country')

I did not go through this code, or the math behind it. The important bit to understand is the coeficient that is attached to each feature name (or word). The function above calculates the odds ratio for a given class based on this coeficient.

In [ ]:
coefs_with_fns = sorted(zip(nb.coef_[0], cv.get_feature_names()))
coefs_with_fns

In [ ]:
#####Ex: Train a classifier on a different two genres. What is the accuracy? What are the most informative features?
#        Do the same with a DTM created using tfidf scores rather than counts.
#        Try it out with three genres. What is the accuracy?
#        Hint: Copy and paste at will. There's no need to write code that has already been written.